ModuleNotFoundError: No module named '__main__.swarm'; '__main__' is not a package

In [4]:
import numpy as np
import timeit


class Swarm:
    def __init__(self, n_particles, dims, c1, c2, w, epochs, obj_func,
                 algo='global', inertia_func='linear', velocity_clamp=None,
                 k=5, u=0.5, m_swarms=3, hueristic_change=0.9, r=5):
        """Intialize the swarm.

        Attributes
        ----------
        n_particles: int
            number of particles in the swarm
        dims: int
            dimensions of the space
        c1: float
            cognitive weight
        c2: float
            social weight
        w: float
            inertia weight
        epochs: int
            number of epochs
        obj_func: string
            name of objective function
        algo: string
            name of swarm algorithm
        intertia_func: string
            name of inertia function
        velocity_clamp: 2-tuple
            minimum velocity (first) and maximum velocity (second)
        k: int
            number of neighbors in a neighborhood
        u: float
            sets local/global ratio in unified (0 to 1)
        m_swarms: int
            number of swarms in 'multiswarm'
        hueristic_change: float
            sets ratio of first/second hueristic in 'multiswarm' (0 to 1)
        r: int
            reshuffle parameter in 'multiswarm'
        """

        self.algo = algo
        self.epochs = epochs
        self.n_particles = n_particles
        self.m_swarms = m_swarms
        self.hueristic_change = hueristic_change
        self.end = False
        self.r = r

        self.runtime = 0

        self.dims = dims
        self.c1 = c1
        self.c2 = c2
        self.w = w
        self.k = k
        self.u = u
        self.decrement = 0.7 / m_swarms

        self.val_min, self.val_max = v_clamp
        self.velocity_bounds = 0.2 * (self.val_max - self.val_min)

        self.obj_func = obj_func
        self.best_cost = float('inf')
        self.best_pos = None
        self.worst_cost = float('inf')
        self.worst_pos = None
        self.local_best_cost = float('inf')
        self.local_best_pos = None

        if self.algo == 'multiswarm':
            self.multiswarm = self.initialize_multiswarm()
        else:
            self.swarm = self.initialize_swarm()

        self.update_best_pos()

    def shape(self):
        return [self.n_particles, self.dims]

    def initialize_swarm(self):
        swarm = []
        for particle in range(self.n_particles):
            swarm.append(Particle(self))
        return swarm

    def initialize_multiswarm(self):
        multiswarm = []
        for _ in self.m:
            m.append(intialize_swarm())
        return multiswarm

    def optimize(self):
        start = timeit.default_timer()
        for i in range(self.epochs):
            if i >= int(self.hueristic_change * self.epochs):
                self.end = True
            if i % self.r == 0:
                self.regroup = True
            for particle in self.swarm:
                particle.update()
                self.update_local_best_pos()
                self.update_global_best_pos()
                self.update_global_worst_pos()
        stop = timeit.default_timer()
        self.runtime = stop - start

    def get_best_neighbor(self, particle):
        distances = []
        for other in self.swarm:
            distances.append((other.pos, other.best_cost, calc_distance(other.pos, particle.pos)))
        sorted_distances = sorted(distances, key=lambda x: x[2])
        return sorted(sorted_distances[:self.k+1], key=lambda x: x[1])

    def update_global_best_pos(self):
        for particle in self.swarm:
            if particle.best_cost < self.best_cost:
                self.best_cost = particle.best_cost
                self.best_pos = particle.best_pos

    def update_local_best_pos(self):
        for particle in self.swarm:
            local_best = get_best_neighbor(particle)
            particle.local_best_cost = local_best[1]
            particle.local_best_pos = local_best[0]


class Particle:
    def __init__(self, swarm):
        self.swarm = swarm
        self.dims = swarm.dims
        self.pos = self.best_pos = np.random.uniform(
            swarm.val_min, swarm.val_max, swarm.dims
        )
        self.velocity = np.random.uniform(
            -swarm.velocity_bounds, swarm.velocity_bounds, swarm.dims
        )
        self.best_cost = swarm.obj_func(self.best_pos)

    def cognitive_weight(self):
        return (self.swarm.c1 * np.random.uniform(0, 1, self.dims)) * (
            self.best_pos - self.pos
        )

    def global_weight(self):
        return (self.swarm.c2 * np.random.uniform(0, 1, self.dims)) * (
            self.swarm.best_pos - self.pos
        )

    def local_weight(self):
        return (self.swarm.c2 * np.random.uniform(0, 1, self.dims)) * (
            self.local_best_pos - self.pos
        )

    def update(self):
        if self.swarm.algo == 'global':
            self.velocity = (self.swarm.w * self.velocity) +(self.cognitive_weight() + self.global_weight())

        if self.swarm.algo == 'local':
            self.velocity = (self.swarm.w * self.velocity) +(self.cognitive_weight() + self.local_weight())

        if self.swarm.algo == 'unified':
            g_velocity = self.u * ((self.swarm.w * self.velocity) + (self.cognitive_weight() + self.global_weight()))
            l_velocity = (1-self.u) * ((self.swarm.w * self.velocity) +(self.cognitive_weight() + self.local_weight()))
            self.velocity = g_velocity + l_velocity

        if self.swarm.algo == 'sa':
            if self.pos == self.swarm.worst_pos:
                new_pos = np.random.uniform(self.swarm.val_min, self.swarm.val_max, self.swarm.dims)
                new_cost = self.swarm.obj_func(new_pos)
                if new_cost < self.cost:
                    self.cost = new_cost
                    self.pos = new_pos
                if self.cost >= self.swarm.alpha:
                    best_neighbor = getNeighbor(self.pos)
                    self.pos = best_neighbor[0]
                    self.cost = best_neighbor[1]

            self.velocity = (self.swarm.w * self.velocity) +(self.cognitive_weight() + self.global_weight())

        if self.swarm.algo == 'multiswarm':
            """Reshuffling"""
            if self.swarm.r:
                particles = [particle for swarm in self.swarm.multiswarm]
                shuffle(particles)
                self.swarm.multiswarm = [particles[i:i+m] for i in range(0, len(particles), m)]

            """Startng heuristic"""
            if not self.swarm.end: 
                self.velocity = (self.swarm.w * self.velocity) +(self.cognitive_weight() + self.local_weight())

            """Ending hueristic"""
            if self.swarm.end:
                self.velocity = (self.swarm.w * self.velocity) +(self.cognitive_weight() + self.global_weight())


        self.pos += self.velocity + self.pos

In [5]:
"""Single objective test functions"""

import numpy as np
import math

def sphere(x):
    return sum([y**2 for y in x])

def rosenbrock(x1, x2):
    return sum([100*((x**2 - y)**2) + (x - 1)**2 for (x, y) in zip(x1,x2)])

def ackley(x):
    return -20 * np.exp(-0.2 * (sum([y**2 for y in x])/len(x) ** 0.5)) - \
          np.exp(sum([np.cos(2 * math.pi * y)/len(x) for y in x])) + 20 + math.e

def griewank(x):
    return sum([y**2 / 4000 for y in x]) - np.prod([np.cos(y/((i+1)**0.5)) for i, y in enumerate(x)]) + 1

def rastrigin(x):
    return sum([y**2 - 10*np.cos(2*math.pi*y) + 10 for y in x])

def weierstrass(x):
    a = 0.5
    b = 3
    k_max = 20

    def sub_sum(x):
        return sum([a**k * np.cos(2*math.pi*(b**k)*(x + 0.5)) for k in range(k_max)])

    val = sum([sub_sum(x0) for x0 in x]) - (len(x) * sum([a**k * np.cos(2*math.pi*(b**k)*0.5) for k in range(k_max)]))

    return val



In [7]:
from .swarm import Swarm

__all__ = ['Swarm']

ModuleNotFoundError: No module named '__main__.swarm'; '__main__' is not a package

In [8]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from setuptools import setup

setup(name='swarmopt',
      version='0.1',
      description='Swarm intelligence hyperparameter optimization.',
      url='http://github.com/siokcronin/swarmopt',
      author='Siobhán K Cronin',
      author_email='siobhankcronin@gmail.com',
      license='MIT',
      keywords='swarmopt',
      packages=['swarmopt'],
      zip_safe=False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.6/distutils/fancy_getopt.py", line 233, in getopt
    opts, args = getopt.getopt(args, short_opts, self.long_opts)
  File "/usr/lib/python3.6/getopt.py", line 95, in getopt
    opts, args = do_shorts(opts, args[0][1:], shortopts, args[1:])
  File "/usr/lib/python3.6/getopt.py", line 195, in do_shorts
    if short_has_arg(opt, shortopts):
  File "/usr/lib/python3.6/getopt.py", line 211, in short_has_arg
    raise GetoptError(_('option -%s not recognized') % opt, opt)
getopt.GetoptError: option -f not recognized

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.6/distutils/core.py", line 134, in setup
    ok = dist.parse_command_line()
  File "/usr/lib/python3.6/distutils/dist.py", line 464, in parse_command_line
    args = parser.getopt(args=self.script_args, object=self)
  File "/usr/lib/python3.6/distutils/fancy_getopt.py", line 235, in getopt
   

TypeError: object of type 'NoneType' has no len()

In [10]:
pip install deap

     |████████████████████████████████| 160 kB 1.9 MB/s eta 0:00:01
You should consider upgrading via the '/home/rupali/gl/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

"""Implementation of the Multiswarm Particle Swarm Optimization algorithm as
presented in *Blackwell, Branke, and Li, 2008, Particle Swarms for Dynamic
Optimization Problems.*
"""

import itertools
import math
import operator
import random

import numpy

try:
    from itertools import imap
except:
    # Python 3 nothing to do
    pass
else:
    map = imap

from deap import base
from deap.benchmarks import movingpeaks
from deap import creator
from deap import tools

scenario = movingpeaks.SCENARIO_2

NDIM = 5
BOUNDS = [scenario["min_coord"], scenario["max_coord"]]

mpb = movingpeaks.MovingPeaks(dim=NDIM, **scenario)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, 
    best=None, bestfit=creator.FitnessMax)
creator.create("Swarm", list, best=None, bestfit=creator.FitnessMax)
        
def generate(pclass, dim, pmin, pmax, smin, smax):
    part = pclass(random.uniform(pmin, pmax) for _ in range(dim)) 
    part.speed = [random.uniform(smin, smax) for _ in range(dim)]
    return part

def convertQuantum(swarm, rcloud, centre, dist):
    dim = len(swarm[0])
    for part in swarm:
        position = [random.gauss(0, 1) for _ in range(dim)]
        dist = math.sqrt(sum(x**2 for x in position))
        
        if dist == "gaussian":
            u = abs(random.gauss(0, 1.0/3.0))
            part[:] = [(rcloud * x * u**(1.0/dim) / dist) + c for x, c in zip(position, centre)]
        
        elif dist == "uvd":
            u = random.random()
            part[:] = [(rcloud * x * u**(1.0/dim) / dist) + c for x, c in zip(position, centre)]
        
        elif dist == "nuvd":
            u = abs(random.gauss(0, 1.0/3.0))
            part[:] = [(rcloud * x * u / dist) + c for x, c in zip(position, centre)]
        
        del part.fitness.values
        del part.bestfit.values
        part.best = None
    
    return swarm

def updateParticle(part, best, chi, c):
    ce1 = (c * random.uniform(0, 1) for _ in range(len(part)))
    ce2 = (c * random.uniform(0, 1) for _ in range(len(part)))
    ce1_p = map(operator.mul, ce1, map(operator.sub, best, part))
    ce2_g = map(operator.mul, ce2, map(operator.sub, part.best, part))
    a = map(operator.sub,
                      map(operator.mul,
                                    itertools.repeat(chi),
                                    map(operator.add, ce1_p, ce2_g)),
                      map(operator.mul,
                                     itertools.repeat(1 - chi),
                                     part.speed))
    part.speed = list(map(operator.add, part.speed, a))
    part[:] = list(map(operator.add, part, part.speed))

toolbox = base.Toolbox()
toolbox.register("particle", generate, creator.Particle, dim=NDIM,
    pmin=BOUNDS[0], pmax=BOUNDS[1], smin=-(BOUNDS[1] - BOUNDS[0])/2.0,
    smax=(BOUNDS[1] - BOUNDS[0])/2.0)
toolbox.register("swarm", tools.initRepeat, creator.Swarm, toolbox.particle)
toolbox.register("update", updateParticle, chi=0.729843788, c=2.05)
toolbox.register("convert", convertQuantum, dist="nuvd")
toolbox.register("evaluate", mpb)

def main(verbose=True):
    NSWARMS = 1
    NPARTICLES = 5
    NEXCESS = 3
    RCLOUD = 0.5    # 0.5 times the move severity

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    logbook = tools.Logbook()
    logbook.header = "gen", "nswarm", "evals", "error", "offline_error", "avg", "max"
    
    # Generate the initial population
    population = [toolbox.swarm(n=NPARTICLES) for _ in range(NSWARMS)]
    
    # Evaluate each particle
    for swarm in population:
        for part in swarm:
            part.fitness.values = toolbox.evaluate(part)

            # Update swarm's attractors personal best and global best
            if not part.best or part.fitness > part.bestfit:
                part.best = toolbox.clone(part[:])          # Get the position
                part.bestfit.values = part.fitness.values   # Get the fitness
            if not swarm.best or part.fitness > swarm.bestfit:
                swarm.best = toolbox.clone(part[:])         # Get the position
                swarm.bestfit.values = part.fitness.values  # Get the fitness

    record = stats.compile(itertools.chain(*population))
    logbook.record(gen=0, evals=mpb.nevals, nswarm=len(population),
                   error=mpb.currentError(), offline_error=mpb.offlineError(), **record)

    if verbose:
        print(logbook.stream)
    
    generation = 1
    while mpb.nevals < 5e5:
        # Check for convergence
        rexcl = (BOUNDS[1] - BOUNDS[0]) / (2 * len(population)**(1.0/NDIM))
        
        not_converged = 0
        worst_swarm_idx = None
        worst_swarm = None
        for i, swarm in enumerate(population):
            # Compute the diameter of the swarm
            for p1, p2 in itertools.combinations(swarm, 2):
                d = math.sqrt(sum((x1 - x2)**2. for x1, x2 in zip(p1, p2)))
                if d > 2*rexcl:
                    not_converged += 1
                    # Search for the worst swarm according to its global best
                    if not worst_swarm or swarm.bestfit < worst_swarm.bestfit:
                        worst_swarm_idx = i
                        worst_swarm = swarm
                    break
        
        # If all swarms have converged, add a swarm
        if not_converged == 0:
            population.append(toolbox.swarm(n=NPARTICLES))
        # If too many swarms are roaming, remove the worst swarm
        elif not_converged > NEXCESS:
            population.pop(worst_swarm_idx)
            
        # Update and evaluate the swarm
        for swarm in population:
            # Check for change
            if swarm.best and toolbox.evaluate(swarm.best) != swarm.bestfit.values:
                # Convert particles to quantum particles
                swarm[:] = toolbox.convert(swarm, rcloud=RCLOUD, centre=swarm.best)
                swarm.best = None
                del swarm.bestfit.values
            
            for part in swarm:
                # Not necessary to update if it is a new swarm
                # or a swarm just converted to quantum
                if swarm.best and part.best:
                    toolbox.update(part, swarm.best)
                part.fitness.values = toolbox.evaluate(part)
                
                # Update swarm's attractors personal best and global best
                if not part.best or part.fitness > part.bestfit:
                    part.best = toolbox.clone(part[:])
                    part.bestfit.values = part.fitness.values
                if not swarm.best or part.fitness > swarm.bestfit:
                    swarm.best = toolbox.clone(part[:])
                    swarm.bestfit.values = part.fitness.values
        
        record = stats.compile(itertools.chain(*population))
        logbook.record(gen=generation, evals=mpb.nevals, nswarm=len(population),
                       error=mpb.currentError(), offline_error=mpb.offlineError(), **record)

        if verbose:
            print(logbook.stream)
        
        # Apply exclusion
        reinit_swarms = set()
        for s1, s2 in itertools.combinations(range(len(population)), 2):
            # Swarms must have a best and not already be set to reinitialize
            if population[s1].best and population[s2].best and not (s1 in reinit_swarms or s2 in reinit_swarms):
                dist = 0
                for x1, x2 in zip(population[s1].best, population[s2].best):
                    dist += (x1 - x2)**2.
                dist = math.sqrt(dist)
                if dist < rexcl:
                    if population[s1].bestfit <= population[s2].bestfit:
                        reinit_swarms.add(s1)
                    else:
                        reinit_swarms.add(s2)
        
        # Reinitialize and evaluate swarms
        for s in reinit_swarms:
            population[s] = toolbox.swarm(n=NPARTICLES)
            for part in population[s]:
                part.fitness.values = toolbox.evaluate(part)
                
                # Update swarm's attractors personal best and global best
                if not part.best or part.fitness > part.bestfit:
                    part.best = toolbox.clone(part[:])
                    part.bestfit.values = part.fitness.values
                if not population[s].best or part.fitness > population[s].bestfit:
                    population[s].best = toolbox.clone(part[:])
                    population[s].bestfit.values = part.fitness.values
        generation += 1

if __name__ == "__main__":
    main()


gen	nswarm	evals	error  	offline_error	avg     	max     
0  	1     	5    	97.9818	111.461      	-81.0052	-47.9818
1  	1     	11   	51.5676	84.5483      	-45.383 	-1.56765
2  	1     	17   	51.5676	72.9081      	-56.1261	-28.1822
3  	1     	23   	50.9016	67.2541      	-20.1711	-0.901566
4  	2     	34   	46.8088	61.4821      	-56.4989	3.19121  
5  	2     	46   	46.8088	57.6543      	-35.9059	2.10527  
6  	2     	58   	46.8088	55.4104      	-56.7255	1.48426  
7  	2     	70   	46.8088	53.9358      	-36.1743	-10.998  
8  	2     	82   	46.8088	52.8928      	-41.9001	-0.717629
9  	2     	94   	42.3055	52.0682      	-45.7823	7.6945   
10 	2     	106  	41.5815	50.9016      	-51.0679	8.41852  
11 	2     	123  	39.9432	49.5069      	-54.5985	10.0568  
12 	2     	140  	29.0509	47.4897      	-41.1456	20.9491  
13 	2     	152  	29.0509	46.034       	-52.6575	18.2084  
14 	2     	164  	29.0509	44.7914      	-38.7702	9.57535  
15 	2     	176  	27.0898	43.629       	-45.0441	22.9102  
16 	2     	188  	2

259	7     	6555 	8.41725    	4.87508      	3.5255    	57.2596  
260	7     	6597 	8.41725    	4.89763      	1.92865   	57.2585  
261	7     	6639 	8.41725    	4.91989      	9.15376   	57.2661  
262	7     	6686 	8.40765    	4.94444      	8.66078   	57.2776  
263	7     	6733 	8.39783    	4.96858      	1.59683   	57.2874  
264	7     	6775 	8.39167    	4.98982      	0.871591  	57.2936  
265	7     	6817 	8.38785    	5.01076      	7.28223   	57.2974  
266	7     	6859 	8.38431    	5.03143      	5.7445    	57.3009  
267	7     	6901 	8.38283    	5.05183      	14.0205   	57.3024  
268	7     	6953 	8.38034    	5.07673      	1.54422   	57.3049  
269	7     	6995 	8.37739    	5.09655      	-3.99551  	57.3078  
270	7     	7037 	8.37739    	5.11613      	8.03865   	57.3059  
271	7     	7084 	8.37556    	5.13776      	4.46847   	57.3097  
272	7     	7131 	8.37145    	5.15909      	-7.39569  	57.3138  
273	7     	7178 	8.36383    	5.18009      	3.84549   	57.3214  
274	7     	7220 	8.36383    	5.19861    

399	7     	12700	9.94725    	7.07185      	29.5125   	56.0697  
400	7     	12742	9.94725    	7.08133      	31.2878   	56.0685  
401	7     	12784	9.94725    	7.09075      	27.7591   	56.0698  
402	7     	12826	9.94596    	7.1001       	35.9608   	56.0714  
403	7     	12868	9.94596    	7.10939      	30.2867   	56.071   
404	7     	12910	9.94557    	7.11862      	35.844    	56.0718  
405	7     	12957	9.94528    	7.12887      	30.8991   	56.0721  
406	7     	12999	9.94457    	7.13797      	30.6305   	56.0728  
407	7     	13041	9.94457    	7.14701      	30.0408   	56.0727  
408	7     	13083	9.94401    	7.15599      	28.3756   	56.0734  
409	7     	13130	9.94401    	7.16597      	36.6908   	56.0731  
410	7     	13172	9.94401    	7.17483      	30.6792   	56.0733  
411	7     	13214	9.94401    	7.18363      	36.956    	56.073   
412	8     	13261	9.94401    	7.19341      	25.4887   	56.0732  
413	8     	13319	9.94362    	7.20539      	24.9694   	56.0738  
414	8     	13372	9.94358    	7.21624    

543	8     	19794	1.08639    	6.15186      	30.4548   	62.8629  
544	8     	19842	1.08112    	6.13959      	29.938    	62.8682  
545	8     	19890	1.08075    	6.12739      	31.3256   	62.8686  
546	8     	19938	1.08075    	6.11524      	33.61     	62.8665  
547	8     	19986	1.08019    	6.10315      	33.2132   	62.8691  
548	8     	20034	3.88622    	6.10196      	30.0288   	59.7304  
549	8     	20082	3.86941    	6.09664      	29.2      	59.7472  
550	8     	20130	3.79729    	6.09123      	32.7082   	59.8193  
551	8     	20178	3.70385    	6.08565      	35.477    	59.9128  
552	8     	20226	3.52177    	6.07976      	33.5212   	60.0949  
553	8     	20274	3.43599    	6.07359      	34.4632   	60.1806  
554	8     	20322	3.41347    	6.06733      	35.5317   	60.2032  
555	8     	20375	3.14375    	6.06007      	32.4489   	60.4729  
556	8     	20423	3.02383    	6.05305      	35.0519   	60.5928  
557	8     	20471	2.97675    	6.04589      	35.304    	60.6399  
558	8     	20519	2.97675    	6.03871    

689	8     	27007	2.24309    	4.97471      	28.4944   	63.3989  
690	8     	27055	2.23772    	4.96986      	34.9277   	63.4042  
691	8     	27108	2.23652    	4.96452      	32.4806   	63.4055  
692	8     	27156	2.21681    	4.95968      	29.8692   	63.4252  
693	8     	27204	2.21042    	4.95483      	36.9162   	63.4315  
694	8     	27252	2.20871    	4.95         	36.3188   	63.4333  
695	8     	27300	2.20871    	4.94518      	37.1519   	63.4327  
696	8     	27348	2.19401    	4.94037      	35.8746   	63.448   
697	8     	27396	2.18899    	4.93555      	40.0071   	63.453   
698	8     	27444	2.18806    	4.93075      	34.8796   	63.4539  
699	8     	27492	2.18683    	4.92596      	38.9153   	63.4551  
700	8     	27540	2.17343    	4.92117      	38.5286   	63.4685  
701	8     	27593	2.17306    	4.91589      	35.36     	63.4689  
702	8     	27646	2.17158    	4.91063      	33.8779   	63.4704  
703	8     	27699	2.17158    	4.90539      	33.0906   	63.4686  
704	8     	27747	2.17158    	4.90066    

...

7715	11    	450454	1.55073    	2.57483      	11.6833   	60.7783  
7716	11    	450520	1.54968    	2.57468      	14.364    	60.7793  
7717	11    	450586	1.549      	2.57453      	21.268    	60.78    
7718	11    	450652	1.54549    	2.57438      	17.9103   	60.7835  
7719	11    	450723	1.54432    	2.57422      	18.8167   	60.7847  
7720	11    	450794	1.54432    	2.57406      	12.3286   	60.7838  
7721	11    	450860	1.5392     	2.57391      	15.9976   	60.7898  
7722	11    	450936	1.53103    	2.57373      	21.3659   	60.798   
7723	11    	451002	1.5296     	2.57358      	6.66743   	60.7994  
7724	11    	451068	1.52501    	2.57343      	19.8379   	60.804   
7725	11    	451144	1.50475    	2.57325      	10.2374   	60.8243  
7726	11    	451210	1.48771    	2.57309      	16.4333   	60.8413  
7727	11    	451281	1.44635    	2.57292      	16.5256   	60.8827  
7728	11    	451347	1.4165     	2.57275      	16.7719   	60.9125  
7729	11    	451418	1.4165     	2.57257      	18.2065   	60.9115  
7730	11   

7936	11    	465665	4.70084    	2.54907      	29.7321   	62.1842  
7937	11    	465736	4.00259    	2.54934      	29.6672   	62.8824  
7938	11    	465807	4.00259    	2.54956      	29.5808   	62.2074  
7939	11    	465878	3.92996    	2.54977      	30.2427   	62.955   
7940	11    	465944	3.92772    	2.54997      	30.9708   	62.9573  
7941	11    	466015	3.78485    	2.55017      	27.9043   	63.1002  
7942	11    	466081	3.69528    	2.55033      	28.9339   	63.1897  
7943	11    	466152	3.69528    	2.55051      	26.0172   	63.1309  
7944	11    	466218	3.69528    	2.55067      	29.9111   	63.1532  
7945	11    	466284	3.58256    	2.55082      	30.5315   	63.3024  
7946	11    	466350	3.5647     	2.55097      	33.4361   	63.3203  
7947	11    	466421	3.5647     	2.55112      	31.4839   	63.2836  
7948	11    	466487	3.5647     	2.55127      	33.5428   	63.3071  
7949	11    	466558	3.5647     	2.55142      	31.2822   	63.3148  
7950	11    	466624	3.55415    	2.55156      	31.1886   	63.3308  
7951	11   

8064	11    	474433	4.36589    	2.57967      	20.5449   	65.5488  
8065	11    	474499	4.36587    	2.57992      	24.7107   	65.5488  
8066	11    	474570	4.36551    	2.58019      	23.6696   	65.5491  
8067	11    	474636	4.36551    	2.58043      	24.1089   	65.5491  
8068	11    	474707	4.36551    	2.5807       	19.8405   	65.5491  
8069	11    	474773	4.36551    	2.58095      	20.0374   	65.5491  
8070	11    	474839	4.36548    	2.5812       	26.0542   	65.5492  
8071	11    	474905	4.36548    	2.58144      	24.6747   	65.5492  
8072	11    	474971	4.36474    	2.58169      	26.9308   	65.5499  
8073	11    	475037	8.10524    	2.58223      	24.5087   	65.5497  
8074	11    	475103	7.93015    	2.58298      	25.6654   	61.145   
8075	11    	475169	7.89292    	2.58372      	23.5685   	61.1822  
8076	11    	475235	7.88936    	2.58446      	24.8769   	61.1858  
8077	11    	475301	7.88071    	2.5852       	27.2266   	61.1944  
8078	11    	475377	7.86131    	2.58604      	20.2286   	61.2138  
8079	11   

8266	11    	488220	0.657482   	2.59391      	18.5204   	64.8514  
8267	11    	488286	0.575604   	2.59364      	17.8819   	64.9333  
8268	11    	488352	0.573233   	2.59337      	25.0147   	64.9356  
8269	11    	488428	0.573233   	2.59305      	23.2619   	64.7702  
8270	11    	488499	0.482944   	2.59275      	21.1823   	65.0259  
8271	11    	488565	0.25454    	2.59245      	22.0703   	65.2543  
8272	11    	488636	0.25454    	2.59211      	19.8445   	65.191   
8273	11    	488702	0.242587   	2.59179      	19.1429   	65.2663  
8274	11    	488768	0.242587   	2.59147      	20.4549   	65.2463  
8275	11    	488839	0.0993303  	2.59112      	17.9393   	65.4095  
8276	11    	488905	0.0993303  	2.59078      	18.2602   	65.3716  
8277	11    	488971	0.0673481  	2.59044      	20.0184   	65.4415  
8278	11    	489037	0.0673481  	2.5901       	18.2757   	65.3994  
8279	11    	489103	0.0673481  	2.58976      	16.1063   	65.4227  
8280	11    	489169	0.0673481  	2.58942      	19.2502   	65.3931  
8281	11   

8390	12    	496713	3.22255    	2.59728      	19.9225   	64.3919  
8391	12    	496785	3.22255    	2.59737      	22.3173   	64.3674  
8392	12    	496857	3.22255    	2.59746      	19.5054   	64.3419  
8393	12    	496929	3.22255    	2.59755      	21.6951   	64.383   
8394	12    	497006	3.13133    	2.59763      	27.696    	64.4831  
8395	12    	497083	3.05743    	2.59771      	24.338    	64.557   
8396	12    	497155	2.97191    	2.59776      	25.5828   	64.6425  
8397	12    	497232	2.8473     	2.5978       	23.0169   	64.7672  
8398	12    	497304	2.72418    	2.59782      	27.5347   	64.8903  
8399	12    	497386	2.68788    	2.59784      	24.4434   	64.9266  
8400	12    	497463	2.47206    	2.59783      	20.6582   	65.1424  
8401	12    	497535	2.29318    	2.59779      	20.6837   	65.3213  
8402	12    	497607	2.2874     	2.59774      	28.7647   	65.3271  
8403	12    	497684	2.15141    	2.59768      	23.7673   	65.463   
8404	12    	497761	1.66084    	2.59755      	23.4594   	65.9536  
8405	12   